# Introduction to JAX

本文目的主要是简单了解JAX的基本情况。

JAX是一种用于**表达**和**组成** 数值程序的**转换** 的语言。JAX还能够编译用于CPU或加速器（GPU / TPU）的数值程序。JAX对于许多数值和科学程序都非常有用，但前提是它们是**在某些约束条件下编写**的。

In [1]:
import numpy as np
from jax import grad, jit
from jax import lax
from jax import random
import jax
import jax.numpy as jnp
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import rcParams
rcParams['image.interpolation'] = 'nearest'
rcParams['image.cmap'] = 'viridis'
rcParams['axes.grid'] = False

### Pure functions

JAX转换和编译仅适用于**功能纯净的Python函数**：所有输入数据均通过函数参数传递，所有结果均通过函数结果输出。如果使用相同的输入调用纯函数，则始终会返回相同的结果。

以下是一些功能上并非纯函数的示例，对于这些函数，JAX的行为不同于Python解释器。注意，JAX系统不能保证这些行为。使用JAX的正确方法是仅在功能上纯的Python函数上使用它。

关于 side-effect of a function in Python，可以参考：[What is a side-effect of a function in Python?](https://dev.to/dev0928/what-is-a-side-effect-of-a-function-in-python-36ei#:~:text=A%20function%20is%20said%20to,gets%20updated%20within%20the%20function.)

In [2]:
def impure_print_side_effect(x):
    print("Executing function")  # This is a side-effect 
    return x

# The side-effects appear during the first run  
print ("First call: ", jit(impure_print_side_effect)(4.))

# Subsequent runs with parameters of same type and shape may not show the side-effect
# This is because JAX now invokes a cached compilation of the function
print ("Second call: ", jit(impure_print_side_effect)(5.))

# JAX re-runs the Python function when the type or shape of the argument changes
print ("Third call, different type: ", jit(impure_print_side_effect)(jnp.array([5.])))

Executing function
First call:  4.0
Second call:  5.0
Executing function
Third call, different type:  [5.]


In [3]:
g = 0.
def impure_uses_globals(x):
    return x + g

# JAX captures the value of the global during the first run
print ("First call: ", jit(impure_uses_globals)(4.))
g = 10.  # Update the global

# Subsequent runs may silently use the cached value of the globals
print ("Second call: ", jit(impure_uses_globals)(5.))

# JAX re-runs the Python function when the type or shape of the argument changes
# This will end up reading the latest value of the global
print ("Third call, different type: ", jit(impure_uses_globals)(jnp.array([4.])))

First call:  4.0
Second call:  5.0
Third call, different type:  [14.]


In [4]:
g = 0.
def impure_saves_global(x):
    global g
    g = x
    return x

# JAX runs once the transformed function with special Traced values for arguments
print ("First call: ", jit(impure_saves_global)(4.))
print ("Saved global: ", g)  # Saved global has an internal JAX value

First call:  4.0
Saved global:  Traced<ShapedArray(float32[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>


即使Python函数实际上在内部使用有状态对象，只要它不读取或写入外部状态，它在功能上也可以是纯函数：

In [5]:
def pure_uses_internal_state(x):
    state = dict(even=0, odd=0)
    for i in range(10):
        state['even' if i % 2 == 0 else 'odd'] += x
    return state['even'] + state['odd']

print(jit(pure_uses_internal_state)(5.))

50.0


不建议在要使用的任何JAX函数jit或任何控制流原语中使用迭代器。原因是迭代器是一个python对象，它引入状态以检索下一个元素。因此，它与JAX功能编程模型不兼容。在下面的代码中，有一些错误尝试将迭代器与JAX一起使用的示例。它们中的大多数返回错误，但有些会产生意外的结果。

```Python
import jax.numpy as jnp
import jax.lax as lax
from jax import make_jaxpr

# lax.fori_loop
array = jnp.arange(10)
print(lax.fori_loop(0, 10, lambda i,x: x+array[i], 0)) # expected result 45
iterator = iter(range(10))
print(lax.fori_loop(0, 10, lambda i,x: x+next(iterator), 0)) # unexpected result 0

# lax.scan
def func11(arr, extra):
    ones = jnp.ones(arr.shape)  
    def body(carry, aelems):
        ae1, ae2 = aelems
        return (carry + ae1 * ae2 + extra, carry)
    return lax.scan(body, 0., (arr, ones))    
make_jaxpr(func11)(jnp.arange(16), 5.)
# make_jaxpr(func11)(iter(range(16)), 5.) # throws error

# lax.cond
array_operand = jnp.array([0.])
lax.cond(True, lambda x: x+1, lambda x: x-1, array_operand)
iter_operand = iter(range(10))
# lax.cond(True, lambda x: next(x)+1, lambda x: next(x)-1, iter_operand) # throws error
```

### In-Place Updates

Numpy的原地更新算法不能使用。如果尝试就地更新JAX设备数组，则会收到错误消息！（☉_☉）

In [6]:
numpy_array = np.zeros((3,3), dtype=np.float32)
print("original array:")
print(numpy_array)

# In place, mutating update
numpy_array[1, :] = 1.0
print("updated array:")
print(numpy_array)

original array:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
updated array:
[[0. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]


In [7]:
jax_array = jnp.zeros((3,3), dtype=jnp.float32)

# In place update of JAX's array will yield an error!
try:
    jax_array[1, :] = 1.0
except Exception as e:
    print("Exception {}".format(e))

Exception '<class 'jaxlib.xla_extension.DeviceArray'>' object does not support item assignment. JAX arrays are immutable; perhaps you want jax.ops.index_update or jax.ops.index_add instead?


这是因为允许就地对变量进行突变会使得程序分析和转换非常困难。JAX需要数值程序的纯函数表达式。

作为替代，JAX提供了其他更新类型的函数：index_update, index_add, index_min, index_max, 和 index.

In [9]:
from jax.ops import index, index_add, index_update

比如，index_update。如果输入值的index_update不重用，JIT -compiled代码将执行这些操作原地

In [10]:
jax_array = jnp.zeros((3, 3))
print("original array:")
print(jax_array)

new_jax_array = index_update(jax_array, index[1, :], 1.)

print("old array unchanged:")
print(jax_array)

print("new array:")
print(new_jax_array)

original array:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
old array unchanged:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
new array:
[[0. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]


比如，index_add。如果输入值的index_update不重用，JIT -compiled代码将执行这些操作原地。

In [11]:
print("original array:")
jax_array = jnp.ones((5, 6))
print(jax_array)

new_jax_array = index_add(jax_array, index[::2, 3:], 7.)
print("new array post-addition:")
print(new_jax_array)

original array:
[[1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]]
new array post-addition:
[[1. 1. 1. 8. 8. 8.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 8. 8. 8.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 8. 8. 8.]]


### Out-of-Bounds Indexing

在Numpy中，通常习惯于在数组的边界之外建立索引时抛出错误，如下所示：

In [12]:
try:
    np.arange(10)[11]
except Exception as e:
    print("Exception {}".format(e))

Exception index 11 is out of bounds for axis 0 with size 10


但是，在其他加速器上引发错误可能会更加困难。JAX就不会引发错误，而是将索引限制在数组的边界上，这意味着对于此示例，将返回数组的最后一个值。

In [13]:
jnp.arange(10)[11]

DeviceArray(9, dtype=int32)

请注意，由于这种行为，jnp.nanargmin和jnp.nanargmax对于由NaN组成的切片返回-1，而Numpy会抛出错误。

### Random Numbers

numpy和其他库中的有状态伪随机数生成器（PRNG）在幕后隐藏了许多细节，提供现成的伪随机数源：

In [14]:
print(np.random.random())
print(np.random.random())
print(np.random.random())

0.4550165972844037
0.8978739797759698
0.335569831356492


numpy使用Mersenne Twister PRNG启动伪随机函数。PRNG的期限为$2^{19937}-1$，可以在任何时候用624个32位无符号整数和一个表示该“熵”已用完的位置描述。

In [17]:
np.random.seed(0)
rng_state = np.random.get_state()
#print(rng_state)
# --> ('MT19937', array([0, 1, 1812433255, 1900727105, 1208447044,
#       2481403966, 4042607538,  337614300, ... 614 more numbers..., 
#       3048484911, 1796872496], dtype=uint32), 624, 0, 0.0)

每当需要一个随机数时，都会在后台自动更新此伪随机状态向量，从而“消耗”梅森扭曲状态向量中的2个uint32：

In [18]:
_ = np.random.uniform()
rng_state = np.random.get_state()
#print(rng_state) 
# --> ('MT19937', array([2443250962, 1093594115, 1878467924,
#       ..., 2648828502, 1678096082], dtype=uint32), 2, 0, 0.0)

# Let's exhaust the entropy in this PRNG statevector
for i in range(311):
    _ = np.random.uniform()
rng_state = np.random.get_state()
#print(rng_state) 
# --> ('MT19937', array([2443250962, 1093594115, 1878467924,
#       ..., 2648828502, 1678096082], dtype=uint32), 624, 0, 0.0)

# Next call iterates the RNG state for a new batch of fake "entropy".
_ = np.random.uniform()
rng_state = np.random.get_state()
# print(rng_state) 
# --> ('MT19937', array([1499117434, 2949980591, 2242547484, 
#      4162027047, 3277342478], dtype=uint32), 2, 0, 0.0)

神奇的PRNG状态的问题在于，很难推断出如何在不同的线程，进程和设备上使用和更新它，并且当最终用户隐藏了熵产生和消耗的细节时，很容易搞砸。

还已知Mersenne Twister PRNG存在一些问题，它具有2.5Kb的状态大小，这导致初始化问题。它无法通过现代BigCrush测试，并且通常很慢。

相反，JAX实现了显式PRNG，通过显式传递和迭代PRNG状态来处理熵的产生和消耗。JAX采用了现代化 Threefry counter-based PRNG。它是可分的，也就是说，它的设计使我们能够将PRNG状态分到新的PRNG中，以用于并行随机生成。

随机状态由两个我们称为键的unsigned-int32描述：

In [19]:
from jax import random
key = random.PRNGKey(0)
key

DeviceArray([0, 0], dtype=uint32)

JAX的随机函数从PRNG状态产生伪随机数，但不改变状态！

重用同一状态将导致悲伤和单调，最终用户失去生命的混乱：

In [20]:
print(random.normal(key, shape=(1,)))
print(key)
# No no no!
print(random.normal(key, shape=(1,)))
print(key)

[-0.20584235]
[0 0]
[-0.20584235]
[0 0]


每当我们需要一个新的伪随机数时，可将PRNG拆分以获取可用的子项：

In [21]:
print("old key", key)
key, subkey = random.split(key)
normal_pseudorandom = random.normal(subkey, shape=(1,))
print("    \---SPLIT --> new key   ", key)
print("             \--> new subkey", subkey, "--> normal", normal_pseudorandom)

old key [0 0]
    \---SPLIT --> new key    [4146024105  967050713]
             \--> new subkey [2718843009 1272950319] --> normal [-1.2515389]


每当需要新的随机数时，传播密钥并创建新的子密钥：

In [22]:
print("old key", key)
key, subkey = random.split(key)
normal_pseudorandom = random.normal(subkey, shape=(1,))
print("    \---SPLIT --> new key   ", key)
print("             \--> new subkey", subkey, "--> normal", normal_pseudorandom)

old key [4146024105  967050713]
    \---SPLIT --> new key    [2384771982 3928867769]
             \--> new subkey [1278412471 2182328957] --> normal [-0.58665067]


一次可以生成多个子键：

In [23]:
key, *subkeys = random.split(key, 4)
for subkey in subkeys:
    print(random.normal(subkey, shape=(1,)))

[-0.37533444]
[0.9864503]
[0.1455319]


### Control Flow

#### ✔python control_flow + autodiff✔

如果只想将grad应用于python函数，则可以正常使用常规python控制流构造，就像在Autograd（或Pytorch或TF Eager）中一样。

In [24]:
def f(x):
    if x < 3:
        return 3. * x ** 2
    else:
        return -4 * x

print(grad(f)(2.))  # ok!
print(grad(f)(4.))  # ok!

12.0
-4.0


#### python控制流+ JIT

控制流和jit一起使用更加复杂，并且默认情况下它具有更多约束。

这样是可以的：

In [25]:
@jit
def f(x):
    for i in range(3):
        x = 2 * x
    return x

print(f(3))

24


这样做也行：

In [26]:
@jit
def g(x):
    y = 0.
    for i in range(x.shape[0]):
        y = y + x[i]
    return y

print(g(jnp.array([1., 2., 3.])))

6.0


但这样做，至少在默认情况下是不行的：

In [27]:
@jit
def f(x):
    if x < 3:
        return 3. * x ** 2
    else:
        return -4 * x

# This will fail!
try:
    f(2)
except Exception as e:
    print("Exception {}".format(e))

Exception Abstract tracer value encountered where concrete value is expected: Traced<ShapedArray(bool[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>
The problem arose with the `bool` function. 
While tracing the function f at <ipython-input-27-99e60ecaa4cc>:1, transformed by jit., this concrete value was not available in Python because it depends on the value of the arguments to f at <ipython-input-27-99e60ecaa4cc>:1, transformed by jit. at flattened positions [0], and the computation of these values is being staged out (that is, delayed rather than executed eagerly).
 (https://jax.readthedocs.io/en/latest/errors.html#jax.errors.ConcretizationTypeError)


为什么这样！？

当jit模式编译一个函数时，我们通常希望编译一个适用于许多不同参数值的函数版本，以便我们可以缓存和重用已编译的代码。这样，我们不必在每个函数求值时都重新编译。

例如，如果我们对针对数组(jnp.array([1., 2., 3.], jnp.float32)的@jit函数求值，我们会希望编译可重复使用的代码来计算，以节省编译时间。

为了获得对许多不同参数值有效的Python代码视图，JAX会在代表可能输入集的**抽象值 abstract value**上跟踪它。存在[多种不同的抽象级别](https://github.com/google/jax/blob/master/jax/abstract_arrays.py)，并且不同的转换使用不同的抽象级别。

默认情况下，jit在ShapedArray抽象级别上跟踪代码，其中每个抽象值代表具有固定形状和dtype的所有数组值的集合。例如，如果我们使用抽象值ShapedArray((3,), jnp.float32)进行跟踪，则会获得该函数的视图，该函数可用于对应数组集合中的任何具体值。这意味着我们可以节省编译时间。

但是这里有一个折衷：如果我们在未承诺特定具体值的ShapedArray((), jnp.float32)上跟踪Python函数，则当我们碰到类似 if x < 3 的行时，表达式 x < 3 的值将表示代表{True, False} 集（即python的set）的抽象ShapedArray((), jnp.bool_)。当Python尝试将其强制转换为具体的True或False时，我们就会收到错误消息：我们不知道采用哪个分支，也无法继续跟踪！折衷方案是，使用更高级别的抽象，我们可以获得更通用的Python代码视图（从而节省了重新编译的时间），但是我们需要对Python代码施加更多约束来完成跟踪。

好消息是可以自己控制这种折衷。通过jit跟踪更精细的抽象值，可以放宽可追溯性约束。例如，使用的static_argnums参数给到jit，我们可以指定跟踪某些参数的具体值。仍然是上面的示例函数，修改为：

In [28]:
def f(x):
    if x < 3:
        return 3. * x ** 2
    else:
        return -4 * x

f = jit(f, static_argnums=(0,))

print(f(2.))

12.0


这是另一个示例，这次涉及一个循环：

In [29]:
def f(x, n):
    y = 0.
    for i in range(n):
        y = y + x[i]
    return y

f = jit(f, static_argnums=(1,))

f(jnp.array([2., 3., 4.]), 2)

DeviceArray(5., dtype=float32)

实际上，循环是静态展开的。JAX还可以跟踪更高的抽象级别，例如Unshaped，但是不是当前任何转换的默认值

️⚠️函数具有参数值依赖的形状

这些控制流问题还会以一种更微妙的方式出现：我们要jit的数值函数不能专门针对参数值内部数组的形状（可以对参数形状进行专门化处理）。举一个简单的例子，创建一个函数，其输出恰好取决于输入变量的length。

In [30]:
def example_fun(length, val):
    return jnp.ones((length,)) * val
# un-jit'd works fine
print(example_fun(5, 4))

bad_example_jit = jit(example_fun)
# this will fail:
try:
    print(bad_example_jit(10, 4))
except Exception as e:
    print("Exception {}".format(e))
# static_argnums tells JAX to recompile on changes at these argument positions:
good_example_jit = jit(example_fun, static_argnums=(0,))
# first compile
print(good_example_jit(10, 4))
# recompiles
print(good_example_jit(5, 4))

[4. 4. 4. 4. 4.]
Exception Shapes must be 1D sequences of concrete values of integer type, got (Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>,).
If using `jit`, try using `static_argnums` or applying `jit` to smaller subfunctions.
[4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
[4. 4. 4. 4. 4.]


如果length在我们的示例中很少更改，static_argnums可以很方便处理，但是如果更改很多，那将是灾难性的！

最后，如果函数具有全局side-effects，那么JAX的跟踪器可能会导致发生奇怪的事情。一个常见的陷阱是尝试在jit函数内部打印数组：

In [31]:
@jit
def f(x):
    print(x)
    y = 2 * x
    print(y)
    return y
f(2)

Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>
Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>


DeviceArray(4, dtype=int32)

### Structured control flow primitives

JAX中有更多控制流选项。假设要避免重新编译，但仍要使用可追溯的控制流，这样可以避免展开大循环。可以使用以下4种结构化的控制流原语：

- lax.cond 可微的
- lax.while_loop fwd-mode可微
- lax.fori_loop fwd-mode可微
- lax.scan 可微的

条件语句的python等效项：

```Python
def cond(pred, true_fun, false_fun, operand):
    if pred:
        return true_fun(operand)
    else:
        return false_fun(operand)
```

In [32]:
from jax import lax

operand = jnp.array([0.])
lax.cond(True, lambda x: x+1, lambda x: x-1, operand)
# --> array([1.], dtype=float32)
lax.cond(False, lambda x: x+1, lambda x: x-1, operand)
# --> array([-1.], dtype=float32)

DeviceArray([-1.], dtype=float32)

while_loop的python等效项：

```Python
def while_loop(cond_fun, body_fun, init_val):
    val = init_val
    while cond_fun(val):
        val = body_fun(val)
    return val
```

In [33]:
init_val = 0
cond_fun = lambda x: x<10
body_fun = lambda x: x+1
lax.while_loop(cond_fun, body_fun, init_val)
# --> array(10, dtype=int32)

DeviceArray(10, dtype=int32)

fori_loop的python等效项：

```Python
def fori_loop(start, stop, body_fun, init_val):
    val = init_val
    for i in range(start, stop):
        val = body_fun(i, val)
    return val
```

In [34]:
init_val = 0
start = 0
stop = 10
body_fun = lambda i,x: x+i
lax.fori_loop(start, stop, body_fun, init_val)
# --> array(45, dtype=int32)

DeviceArray(45, dtype=int32)

小结下：

|construct|jit|grad|
|-|-|-|
|if|❌|✔|
|for|✔∗|✔|
|while|✔∗|✔|
|lax.cond|✔|✔|
|lax.while_loop|✔|fwd|
|lax.fori_loop|✔|fwd|
|lax.scan|✔|✔|

### NaNs

调试NaN:如果要跟踪函数或梯度中NaN的发生位置，可以通过以下方法打开NaN检查器：

- 设置JAX_DEBUG_NANS=True环境变量；
- 添加from jax.config import config和config.update("jax_debug_nans", True)并靠近主文件的顶部；
- 加入from jax.config import config和config.parse_flags_with_absl()到你的主文件，然后使用命令行标志，比如--jax_debug_nans=True，设置选项;

这将导致计算在生成NaN时立即出错。启用此选项会为XLA生成的每个浮点类型值添加一个Nan校验。这意味着将值拉回主机，并针对不在@jit之下的每个基本操作，将其作为ndarray进行检查。对于在@jit下的代码，将检查每个@jit函数的输出，如果存在nan，它将以非优化的逐操作模式重新运行该函数，从而有效地一次删除@jit的一个级别。

可能会出现一些棘手的情况，例如仅在一个@jit而不在非优化模式下生成的nan 。在这种情况下，您会看到一条警告消息，但您的代码将继续执行。

如果在梯度求值的向后传递中生成nans，则在堆栈跟踪中向上引发几帧异常时，将使用向后传递函数，该函数本质上是一个简单的jaxpr解释器，用于遍历原始操作的序列。

In [35]:
import jax.numpy as jnp
jnp.divide(0., 0.)

DeviceArray(nan, dtype=float32)

生成的nan被捕捉到了。通过运行%debug，我们可以获取事后调试器。这也适用于@jit下的函数，如下所示。

In [36]:
from jax import jit

@jit
def f(x, y):
    a = x * y
    b = (x + y) / (x - y)
    c = a + 2
    return a + b * c

x = jnp.array([2., 0.])
y = jnp.array([3., 0.])
f(x, y)

DeviceArray([-34.,  nan], dtype=float32)

当此代码在@jit函数的输出中看到nan时，它将调用未优化的代码，因此我们仍然可以获得清晰的堆栈跟踪。然后，我们可以运行验尸调试器%debug来检查所有值以找出错误。

⚠️如果不进行调试，则不应该启用NaN检查器，因为它可能会引入很多设备主机往返和性能下降！

最后注意双精度（64位）。目前，JAX默认情况下会强制使用单精度数字，以减轻Numpy API积极将操作数提升为double的趋势。这是许多机器学习应用程序所期望的行为！

In [37]:
x = random.uniform(random.PRNGKey(0), (1000,), dtype=jnp.float64)
x.dtype

dtype('float32')